<a href="https://colab.research.google.com/github/tesseract314/DS-Unit-4-Sprint-2-NLP/blob/master/LS_DS_424_Word_Embeddings_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 2 Assignment 4*

# Word Embeddings

### Use Word2Vec to train your own model on a dataset.

1) **Optional** - Find your own dataset of documents to train you model on. You are going to need a lot of data, so it's probably not realistic to scrape data for this assignment given the time constraints that we're working under. Try to find a dataset that has > 5000 documents.

- If you can't find a dataset to use try this one: <https://www.kaggle.com/c/quora-question-pairs>

2) Clean/Tokenize the documents.

3) Vectorize the model using Word2Vec and explore the results using each of the following at least one time:

- your_model.wv.most_similar()
- your_model.wv.similarity()
- your_model.wv.doesn't_match()

In [0]:
import nltk
nltk.download('all')

In [0]:
import gensim
import pandas as pd
from nltk.tokenize import word_tokenize
from gensim.models.word2vec import Word2Vec
from nltk.corpus import stopwords
import string

pd.set_option('max_colwidth', 600)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [41]:
df = pd.read_csv('train.csv')
df.head(10)

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in share market in india?,What is the step by step guide to invest in share market?,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Diamond?,What would happen if the Indian government stole the Kohinoor (Koh-i-Noor) diamond back?,0
2,2,5,6,How can I increase the speed of my internet connection while using a VPN?,How can Internet speed be increased by hacking through DNS?,0
3,3,7,8,Why am I mentally very lonely? How can I solve it?,"Find the remainder when [math]23^{24}[/math] is divided by 24,23?",0
4,4,9,10,"Which one dissolve in water quikly sugar, salt, methane and carbon di oxide?",Which fish would survive in salt water?,0
5,5,11,12,Astrology: I am a Capricorn Sun Cap moon and cap rising...what does that say about me?,"I'm a triple Capricorn (Sun, Moon and ascendant in Capricorn) What does this say about me?",1
6,6,13,14,Should I buy tiago?,What keeps childern active and far from phone and video games?,0
7,7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1
8,8,17,18,When do you use シ instead of し?,"When do you use ""&"" instead of ""and""?",0
9,9,19,20,Motorola (company): Can I hack my Charter Motorolla DCX3400?,How do I hack Motorola DCX3400 for free internet?,0


In [0]:
# clean function
def clean_doc(doc):
    # split into tokens by white space
    tokens = doc.split()
    # remove punctuation from each token
    table = str.maketrans('', '', string.punctuation)
    tokens = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # filter out stop words
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
    # filter out short tokens
    tokens = [word.lower() for word in tokens if len(word) > 1]
    return tokens

In [0]:
# tokenize column
df['question1'] = df['question1'].astype(str).apply(clean_doc)

In [0]:
df['question1'].head(40)

In [0]:
model = Word2Vec(df['question1'], min_count=5, size=300, window=2)

In [86]:
model.wv.most_similar('stuff')

[('fetish', 0.7076653242111206),
 ('clothes', 0.6959240436553955),
 ('anymore', 0.6464889049530029),
 ('liars', 0.6443381309509277),
 ('cockroaches', 0.6439735889434814),
 ('groceries', 0.6315992474555969),
 ('crackers', 0.6290179491043091),
 ('hoarding', 0.6283577680587769),
 ('khaki', 0.6248286962509155),
 ('nite', 0.6238281726837158)]

In [87]:
model.wv.similarity('life', 'death')

0.32940596

In [90]:
model.wv.similarity('up', 'down')

0.51087546

In [103]:
print(int(model.wv.similarity('nazi', 'communist')*100),'%', 'Similarity')

73 % Similarity


### Stretch Goals:

1) Use Doc2Vec to train a model on your dataset, and then provide model with a new document and let it find similar documents.

2) Download the pre-trained word vectors from Google. Access the pre-trained vectors via the following link: https://code.google.com/archive/p/word2vec

  - Load the pre-trained word vectors and train the Word2vec model

  - Examine the first 100 keys or words of the vocabulary

  - Outputs the vector representation for a select set of words - the words can be of your choice

  - Examine the similarity between words - the words can be of your choice

  - For example:

    - model.similarity('house', 'bungalow')

    - model.similarity('house', 'umbrella')
    
3) Word2Vec is not the latest or greatest wording embedding model, but it has been around long enough to have well developed APIs in several of the major packages. Try one of the State of the Art (SoA) embeddings models on your dataset. The state of the art models are BERT and Elmo (and yes, that is a pun). 